In [1]:
# 判断是否有糖尿病
# 自我报告或者根据空腹血糖和oggt实验得出
### DIQ010          自我报告是否有糖尿病
### LBDGLUSI        空腹血糖
### LBDGLTSI        oggt2小时血糖
### diabetes_level  糖尿病等级  0没有 1有 2糖尿病前期

import pandas as pd
import numpy as np

In [2]:
# 提取自我报告是否有糖尿病的数据 - diabetes_extract_df
diabetes_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Diabetes/DIQ_F.XPT')
diabetes_extract_features = [
    'SEQN',
    'DIQ010',           #是否有糖尿病
]
diabetes_extract_df = diabetes_df[diabetes_extract_features]
diabetes_extract_df['DIQ010'].replace([7,9], np.nan, inplace=True)

C:\Users\Gstar\AppData\Local\Temp\ipykernel_15740\2510475123.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diabetes_extract_df['DIQ010'].replace([7,9], np.nan, inplace=True)


In [3]:
# 提取oggt的血糖结果 - ogtt_extract_df
ogtt_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Diabetes/OGTT_F.XPT')

ogtt_extract_features = [
    'SEQN',
    'LBDGLTSI',         #两小时血糖(mmol/L)
]
ogtt_extract_df = ogtt_df[ogtt_extract_features]

In [4]:
# 提取空腹血糖的数据 - fasting_glucose_df
fasting_glucose_df = pd.read_sas('E:/work/Paper/NHANES/0910Analysis Data/Diabetes/GLU_F.XPT')

fasting_glucose_extract_features = [
    'SEQN',
    'LBDGLUSI',   #空腹血糖(mmol/L)
]
fasting_glucose_extract_df = fasting_glucose_df[fasting_glucose_extract_features]

In [5]:
# 合并上述数据 - diabetes_df
diabetes_df = pd.merge(left=fasting_glucose_extract_df, right=ogtt_extract_df, on='SEQN', how='outer')
diabetes_df = pd.merge(left=diabetes_extract_df, right=diabetes_df, on='SEQN', how='outer')

In [6]:
# 判断糖尿病
### 0 没有
### 1 有
### 2 糖尿病前期
def judge_diabetes(x):
    if x['DIQ010'] == 1:
        judge = 1
    elif (pd.isna(x['DIQ010'])) | (x['DIQ010']==9 )| (x['DIQ010']== 7):
        if pd.isna(x['LBDGLUSI']) | (pd.isna(x['LBDGLTSI'])):
            judge = pd.NA
        else:
            if (x['LBDGLUSI'] < 6.1) | (x['LBDGLTSI'] < 7.8):
                judge = 0
            elif (6.1 < x['LBDGLUSI'] < 6.9) | (x['LBDGLTSI'] < 7.8):
                judge = 2   #糖尿病前期
            elif (x['LBDGLUSI'] < 7) | (7.8 < x['LBDGLTSI'] < 11.0):
                judge = 2   #糖尿病前期
            elif (x['LBDGLUSI'] >= 7) | (7.8 < x['LBDGLTSI'] >= 11.1):
                judge = 1
    else:
        if x['DIQ010'] == 2:
            judge = 0
        elif x['DIQ010'] == 3:
            judge = 2
        if (x['LBDGLUSI'] < 6.1) | (x['LBDGLTSI'] < 7.8):
            judge = 0
        elif (6.1 < x['LBDGLUSI'] < 6.9) | (x['LBDGLTSI'] < 7.8):
            judge = 2   #糖尿病前期
        elif (x['LBDGLUSI'] < 7) | (7.8 < x['LBDGLTSI'] < 11.0):
            judge = 2   #糖尿病前期
        elif (x['LBDGLUSI'] >= 7) | (7.8 < x['LBDGLTSI'] >= 11.1):
            judge = 1
    return judge

diabetes_df['diabetes_level'] = diabetes_df.apply(judge_diabetes, axis=1)

In [7]:
diabetes_df.to_excel('E:/work/Paper/NHANES/0910Analysis Data/Final data/diabetes.xlsx')